In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch import nn
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
features = ["Sex", "Age", "Survived"]
train_df = train_df[features]

frequency_table = pd.crosstab(train_df['Sex'], train_df['Survived'])
frequency_table.plot(kind='bar', stacked=True, figsize=(8, 6))
plt.xlabel('Gender')
plt.ylabel('Count')
plt.title('Survival Count by Gender')
plt.legend(title='Survived', labels=['No', 'Yes'])
plt.show()

In [ ]:
train_df = pd.get_dummies(train_df).sample(frac = 1).dropna()
train_split = int(0.8*len(train_df))
test_df = train_df.iloc[train_split:,:]
train_df = train_df.iloc[:train_split,:]
x_train_df, y_train_df = train_df.drop(['Survived', 'Sex_male'] , axis = 1), train_df['Survived']
x_test_df, y_test_df = test_df.drop(['Survived', 'Sex_male'] , axis = 1), test_df['Survived']
x_train_df

In [ ]:
X_train, y_train = torch.tensor(x_train_df.values.astype('float32')), torch.tensor(y_train_df.values.astype('float32'))
X_test, y_test = torch.tensor(x_test_df.values.astype('float32')), torch.tensor(y_test_df.values.astype('float32'))
X_train

In [ ]:
class Classification(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2, 15)
        self.layer2 = nn.Linear(15, 1)
 
    def forward(self, x):
        x = self.layer2(self.layer1(x))
        return x

model_1 = Classification()

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(params=model_1.parameters(),
                            lr=0.01)

In [ ]:
epochs = 100000

for epoch in range(epochs):

    model_1.train()
    y_logits = model_1(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))
    loss = loss_fn(y_logits, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    model_1.eval()
    with torch.inference_mode():
        test_logits = model_1(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))
        
        test_loss = loss_fn(test_logits, y_test)
    if epoch % 10000 == 0:
         print(f"Epoch: {epoch} | Loss: {loss:.5f} | Test loss: {test_loss:.5f}")

In [ ]:
if "Survived" in features: 
    features.remove("Survived")
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
test_d = pd.get_dummies(test_df[features]).sample(frac = 1).drop(['Sex_male'], axis = 1)
for x in test_d.keys():
    mean = test_d[x].mean()
    test_d[x].fillna(mean, inplace=True)
model_1.eval()
test = torch.tensor(test_d.values.astype('float32'))
with torch.inference_mode():
    test_logits = model_1(test).squeeze()
    test_preds = torch.round(torch.sigmoid(test_logits))
output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': test_preds})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")